In [25]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/moons/many"
results_dir_relu = "results/classification/single_layer/relu/moons"
results_dir_tanh = "results/classification/single_layer/tanh/moons"
#model_names_relu = ["Dirichlet Horseshoe"]
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [ ]:
data_dir = f"datasets/moons"
results_dir_relu = "results/classification/single_layer/relu/moons"
results_dir_tanh = "results/classification/single_layer/tanh/moons"
#model_names_relu = ["Dirichlet Horseshoe"]
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [28]:
import re

from utils.generate_data import generate_two_moons
from utils.robust_utils import build_pytorch_model_from_stan_sample
def evaluate_on_generated_test_set(
    fits_dict,
    input_dim,
    hidden_dim,
    output_dim,
    N_test=1000,
    sigma=0.2,
    D=2,
    seed=123,
    sample_indices=range(0, 2000, 50),
):
    import torch
    import torch.nn.functional as F
    from sklearn.metrics import accuracy_score, log_loss

    _, X_test, _, y_test = generate_two_moons(N=N_test, sigma=sigma, test_size=0.99, D=D, seed=seed)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test = np.asarray(y_test)

    results = {}

    for model_name, model_entry in fits_dict.items():
        fit = model_entry["posterior"]

        probs = []

        for idx in sample_indices:
            model = build_pytorch_model_from_stan_sample(
                fit=fit,
                sample_idx=idx,
                input_dim=input_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                task="classification"
            )
            model.eval()
            with torch.no_grad():
                logits = model(X_test_tensor)
                prob = F.softmax(logits, dim=1).numpy()
                probs.append(prob)

        probs = np.stack(probs, axis=0)  # shape: (n_samples, N_test, 2)
        mean_probs = np.mean(probs, axis=0)
        y_pred_mode = np.argmax(mean_probs, axis=1) + 1

        acc = accuracy_score(y_test, y_pred_mode)
        nll = log_loss(y_test - 1, mean_probs)

        results[model_name] = {
            "accuracy": acc,
            "nll": nll
        }
    return results, X_test, y_test

def extract_metadata_from_key(key):
    """Hent ut N og seed fra nøkkel som 'Moon_N100_p2_sigma0.20_seed1'"""
    match = re.search(r"N(\d+)_p\d+_sigma[\d\.]+_seed(\d+)", key)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        return None, None

def evaluate_all_models_on_generated_test_set(fits_dict, N_test=200, sample_indices=range(2000)):
    all_results = []

    for key, fits in fits_dict.items():
        N, seed = extract_metadata_from_key(key)
        if N is None:
            continue

        result, _, _ = evaluate_on_generated_test_set(
            fits_dict=fits,
            input_dim=2,
            hidden_dim=16,
            output_dim=2,
            N_test=N_test,
            sigma=0.2,
            D=2,
            seed=999,  # fast seed for fair test-set
            sample_indices=sample_indices
        )

        for model_name, metrics in result.items():
            all_results.append({
                "config": key,
                "model": model_name,
                "N": N,
                "seed": seed,
                "accuracy": metrics["accuracy"],
                "nll": metrics["nll"]
            })

    df = pd.DataFrame(all_results)
    return df


In [33]:
df_results = evaluate_all_models_on_generated_test_set(relu_fits, N_test=500)

In [ ]:
summary = df_results.groupby(["model", "N"]).agg(
    acc_mean=("accuracy", "mean"),
    acc_std=("accuracy", "std"),
    nll_mean=("nll", "mean"),
    nll_std=("nll", "std"),
).reset_index()

print(summary)


In [ ]:
print(summary.to_latex(index=False, float_format="%.3f"))

In [ ]:
sns.pointplot(data=df_results, x="N", y="accuracy", hue="model", dodge=True, errorbar="sd")
plt.title("Accuracy on generated test set")
plt.show()

sns.pointplot(data=df_results, x="N", y="nll", hue="model", dodge=True, errorbar="sd")
plt.title("Negative log-likelihood on generated test set")
plt.show()
